## Output & Integration Instructions

This notebook generates `trained_model_weights.json`, which is used by the
client-side web application for real-time inference.

### How to integrate with the UI
1. Run the cell below.
2. Download the generated `trained_model_weights.json`.
3. Open `index.html`.
4. Locate:
   ```js
   const TRAINED_MODEL = { ... };
5. Replace the contents with the generated JSON data.

In [ ]:
import pandas as pd
import json
import ast
import csv
try:
    from tqdm.notebook import tqdm
except ImportError:
    from tqdm import tqdm

# --- CONFIGURATION ---
# 1. We force the timezone to ensure "9 AM" means "9 AM Local Time"
TARGET_TIMEZONE = 'Asia/Kolkata'
# 2. We set a HIGH threshold. An app must account for 5% of daily usage
#    in a specific hour to be considered "Active" in that hour.
USAGE_THRESHOLD = 0.05

print("--- [ MODULE 3 ] FINAL STRICTER TRAINING ---")

# --- 1. ROBUST DATA LOADING ---
def parse_apps_string(apps_str):
    if pd.isna(apps_str): return []
    try:
        return ast.literal_eval(apps_str)
    except (ValueError, SyntaxError):
        try:
            return json.loads(apps_str.replace("'", "\""))
        except:
            return []

def get_simple_app_name(process_name):
    try:
        process_name = str(process_name)
        parts = process_name.split('.')
        if len(parts) > 1:
            if parts[0] == 'com' and len(parts) > 2:
                name = parts[1]
                if name in ['android', 'google', 'samsung', 'htc', 'motorola']:
                     if len(parts) > 3: return parts[3]
                     elif len(parts) > 2: return parts[2]
                return name
            else:
                return parts[0]
        return process_name
    except:
        return 'unknown'

try:
    expected_columns = [
        'uuid', 'timestamp', 'batteryLevel', 'batteryStatus',
        'timeZone', 'mobileCountryCode', 'apps'
    ]
    data_list = []

    print("Loading CSV...")
    with open('mega_dataset.csv', 'r', encoding='utf-8') as f:
        reader = csv.reader(f, quotechar='"')
        next(reader)
        for row in reader:
            if len(row) == len(expected_columns):
                row_dict = dict(zip(expected_columns, row))
                data_list.append(row_dict)

    df_raw = pd.DataFrame(data_list)
    print("Parsing app lists...")
    df_raw['apps_list'] = df_raw['apps'].apply(parse_apps_string)

    print("Exploding data...")
    df_exploded = df_raw.explode('apps_list')
    df_exploded = df_exploded.dropna(subset=['apps_list'])
    df_exploded = df_exploded[df_exploded['apps_list'].apply(lambda x: isinstance(x, dict))]

    print("Normalizing...")
    app_data = pd.json_normalize(df_exploded['apps_list'])
    df_exploded = df_exploded.reset_index(drop=True)
    app_data = app_data.reset_index(drop=True)
    df_clean = pd.concat([df_exploded, app_data], axis=1)

    # --- TIMEZONE NORMALIZATION ---
    print(f"Normalizing Timestamps to {TARGET_TIMEZONE}...")

    # 1. Fallback check
    if TARGET_TIMEZONE not in df_clean['timeZone'].unique():
        print(f"⚠️ Warning: No data found for {TARGET_TIMEZONE}.")
        dominant_tz = df_clean['timeZone'].mode()[0]
        print(f"🔄 Switching to dominant timezone: {dominant_tz}")
        training_tz = dominant_tz
    else:
        training_tz = TARGET_TIMEZONE

    # 2. Filter & Convert
    # We take the data for the target timezone
    df_final = df_clean[df_clean['timeZone'] == training_tz].copy()

    # 3. Proper Datetime Conversion
    df_final['timestamp'] = pd.to_numeric(df_final['timestamp'], errors='coerce')
    df_final['datetime'] = pd.to_datetime(df_final['timestamp'], unit='s')

    # Convert UTC to Local Time
    df_final['local_time'] = df_final['datetime'].dt.tz_localize('UTC').dt.tz_convert(training_tz)
    df_final['hour_of_day'] = df_final['local_time'].dt.hour

    df_final['simpleAppName'] = df_final['processName'].apply(get_simple_app_name)

    print(f"Training Data Ready: {len(df_final)} rows from {training_tz}")

except Exception as e:
    print(f"FATAL ERROR: {e}")
    raise e

# --- 2. TRAINING (STRICT LOGIC) ---
print("\n--- LEARNING PATTERNS ---")

# Apps we want to track for the "Resource Allocation" feature
target_apps = ['youtube', 'instagram', 'chrome', 'calendar', 'clock', 'facebook', 'snapchat', 'whatsapp', 'spotify', 'gm', 'maps', 'twitter']
usage_map = {}

print(f"Applying strict {USAGE_THRESHOLD*100}% usage threshold...")

for app in target_apps:
    # Only look at FOREGROUND (Active) usage
    app_usage = df_final[
        (df_final['simpleAppName'].str.contains(app, case=False)) &
        (df_final['priority'] == 'Foreground app')
    ]

    if not app_usage.empty:
        # Calculate: For this specific app, what % of usage happens in each hour?
        hourly_distribution = app_usage['hour_of_day'].value_counts(normalize=True)

        # Filter: Keep ONLY hours that have > 5% of the total usage
        significant_hours = hourly_distribution[hourly_distribution > USAGE_THRESHOLD].index.tolist()
        usage_map[app] = sorted(significant_hours)
    else:
        usage_map[app] = []

# --- [ NEW ] COMMON SENSE FILTER ---
# We enforce logical constraints to ensure the recommendations make sense
print("Applying Common Sense Constraints...")

# Rule 1: Clock is primarily used for alarms (Night/Morning)
if 'clock' in usage_map:
    # Force include late night and early morning hours
    common_sense_clock = [22, 23, 5, 6, 7]
    usage_map['clock'] = sorted(list(set(usage_map['clock'] + common_sense_clock)))

# Rule 2: Calendar is primarily a daytime productivity tool
if 'calendar' in usage_map:
    # Force include standard work hours
    common_sense_calendar = [8, 9, 10, 11, 14, 15]
    usage_map['calendar'] = sorted(list(set(usage_map['calendar'] + common_sense_calendar)))

# Rule 3: News/Weather (e.g. Chrome/Google) often checked in morning
if 'chrome' in usage_map:
    common_sense_morning = [7, 8, 9]
    usage_map['chrome'] = sorted(list(set(usage_map['chrome'] + common_sense_morning)))

print(f"Refined Usage Map: {json.dumps(usage_map, indent=2)}")

# --- 3. GENERATING BLACKLIST ---
print("\n--- GENERATING BLACKLIST ---")
background_apps = []
all_apps_counts = df_final['simpleAppName'].value_counts().head(50)

for app_name in all_apps_counts.index:
    app_data = df_final[df_final['simpleAppName'] == app_name]
    # Ratio of Foreground vs. Total
    fg_ratio = len(app_data[app_data['priority'] == 'Foreground app']) / len(app_data)

    # If app is popular (>50 events) but almost NEVER in foreground (<5%)
    if len(app_data) > 50 and fg_ratio < 0.05:
        background_apps.append(app_name)

print(f"Blacklist generated: {len(background_apps)} apps found.")

# --- 4. EXPORTING MODEL ---
trained_model = {
    "usage_map": usage_map,
    "connectivity": { "home_timezone": training_tz, "home_mcc": "unknown" },
    "blacklisted_apps": background_apps
}

filename = 'trained_model_weights.json'
with open(filename, 'w') as f:
    json.dump(trained_model, f, indent=4)

print("\n================================================")
print(f"SUCCESS! Model saved to '{filename}'")
print("================================================")
